In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content/drive/MyDrive/Pickleapp/224x224_sized')

In [ ]:
X_train = np.load("train_X.npy")
y_train = np.load("train_y.npy")
X_test = np.load("test_X.npy")
y_test = np.load("test_y.npy")
X_test_noaug = np.load("noaug_test_X.npy")
y_test_noaug = np.load("noaug_test_y.npy")
right_X_train = np.load("right_orient_train_X.npy")
right_y_train = np.load("right_orient_train_y.npy")
right_X_test = np.load("right_orient_test_X.npy")
right_y_test = np.load("right_orient_test_y.npy")


In [ ]:
shape = (224, 224, 3)
points = [1,2,3,4,5,6]
point_cols = [(2 * (p-1), 2 * (p-1) + 1) for p in points]
point_cols = np.array(point_cols).flatten()
num_points = len(points) * 2

y_train = y_train[:, point_cols]
y_test = y_test[:, point_cols]
y_test_noaug = y_test_noaug[:, point_cols]
right_y_train = right_y_train[:, point_cols]
right_y_test = right_y_test[:, point_cols]


In [ ]:
bm = tf.keras.applications.MobileNetV3Small(
    include_top=False,
    weights="imagenet",
    input_shape=shape,
)
bm.trainable=False

4334752/4334752 [==============================] - 1s 0us/step


In [ ]:
inputs = tf.keras.Input(shape=shape)
x = bm(inputs, training=False)
# x.trainable = False
# x = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(num_points, activation = 'relu')(x)
model = tf.keras.Model(inputs=inputs, outputs=x)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 MobilenetV3small (Function  (None, 7, 7, 576)         939120    
 al)                                                             
                                                                 
 flatten (Flatten)           (None, 28224)             0         
                                                                 
 dropout (Dropout)           (None, 28224)             0         
                                                                 
 dense (Dense)               (None, 12)                338700    
                                                                 
Total params: 1277820 (4.87 MB)
Trainable params: 338700 (1.29 MB)
Non-trainable params: 939120 (3.58 MB)
_____________________

In [ ]:
def average_distance_loss(y_true, y_pred):
    """
    Custom loss function to calculate the average distance between predicted and target coordinates for 6 points.

    Parameters:
    - y_true: Tensor of shape (batch_size, 12), representing the target coordinates.
    - y_pred: Tensor of shape (batch_size, 12), representing the predicted coordinates.

    Returns:
    - loss: Tensor representing the average distance loss.
    """
    # Reshape the tensors to (batch_size, 6, 2) for easy indexing of coordinates
    # print(y_true)
    # print(y_pred)
    y_true = tf.reshape(y_true, (-1, len(points), 2))
    y_pred = tf.reshape(y_pred, (-1, len(points), 2))

    # Calculate Euclidean distance between each pair of coordinates
    distances = tf.norm(y_true - y_pred, axis=-1)
    # print(distances)
    # Calculate the average distance across all coordinates and batches
    loss = tf.reduce_mean(distances)

    return loss

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                loss=average_distance_loss,
                metrics=['mae'])

In [ ]:
print(len(right_X_train), len(right_X_test))

714 186


In [ ]:
callback = tf.keras.callbacks.EarlyStopping('val_loss', patience=10, restore_best_weights = True)
np.random.seed(42)
model.fit(right_X_train, right_y_train, epochs = 400, batch_size = 32, validation_data=(right_X_test, right_y_test), callbacks = [callback])

Epoch 1/400
23/23 [==============================] - 18s 192ms/step - loss: 1246.0977 - mae: 821.9709 - val_loss: 1241.0015 - val_mae: 817.5974
Epoch 2/400
23/23 [==============================] - 1s 33ms/step - loss: 1212.1492 - mae: 796.4379 - val_loss: 1207.2361 - val_mae: 792.2004
Epoch 3/400
23/23 [==============================] - 1s 35ms/step - loss: 1178.2542 - mae: 770.9127 - val_loss: 1173.9161 - val_mae: 767.0912
Epoch 4/400
23/23 [==============================] - 1s 33ms/step - loss: 1144.8848 - mae: 745.7234 - val_loss: 1140.9382 - val_mae: 742.2120
Epoch 5/400
23/23 [==============================] - 1s 35ms/step - loss: 1111.8793 - mae: 720.8242 - val_loss: 1108.3262 - val_mae: 717.5986
Epoch 6/400
23/23 [==============================] - 1s 30ms/step - loss: 1079.0540 - mae: 696.0189 - val_loss: 1076.0071 - val_mae: 693.1924
Epoch 7/400
23/23 [==============================] - 1s 30ms/step - loss: 1046.6382 - mae: 671.5228 - val_loss: 1043.9039 - val_mae: 668.9480
Epoc

In [ ]:
# plot the history of the loss
start = 1
loss = model.history.history['loss']
val_loss = model.history.history['val_loss']
plt.plot(loss[start:], label="loss")
plt.plot(val_loss[start:], label='val loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.legend()


In [ ]:
# test
model.evaluate(right_X_test, right_y_test)

In [ ]:
# training
model.evaluate(right_X_train, right_y_train)

In [ ]:
# unaugmented all test data
model.evaluate(X_test_noaug, y_test_noaug)

In [ ]:
bm.trainable = True
model.summary()

In [ ]:
# save weights
from google.colab import files
filename = 'MobileNetV3Small_right_only.h5'
model.save_weights(filename)
files.download(filename)